In [9]:
from __future__ import print_function
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import add, Activation, LSTM, Conv1D
import tensorflow as tf
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import numpy as np
import csv
import h5py
import time
import os
import shutil

import sys
sys.path.append('/Users/hao/opt/anaconda3/envs/eqtdev/lib/python3.7/site-packages/EQTransformer/core')

from EqT_utils import f1, SeqSelfAttention, FeedForward, LayerNormalization
from EqT_utils import generate_arrays_from_file, picker
from EqT_utils import DataGeneratorTest, PreLoadGeneratorTest
np.warnings.filterwarnings('ignore')
import datetime
from tqdm import tqdm
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False



In [7]:
#import default args
input_hdf5=None
input_testset=None
input_model=None
output_name=None
detection_threshold=0.20
P_threshold=0.1
S_threshold=0.1
number_of_plots=100
estimate_uncertainty=True
number_of_sampling=5
loss_weights=[0.05, 0.40, 0.55]
loss_types=['binary_crossentropy', 'binary_crossentropy','binary_crossentropy']
input_dimention=(6000, 3)
normalization_mode='std'
mode='generator'
batch_size=500
gpuid=None
gpu_limit=None

args = {
"input_hdf5": input_hdf5,
"input_testset": input_testset,
"input_model": input_model,
"output_name": output_name,
"detection_threshold": detection_threshold,
"P_threshold": P_threshold,
"S_threshold": S_threshold,
"number_of_plots": number_of_plots,
"estimate_uncertainty": estimate_uncertainty,
"number_of_sampling": number_of_sampling,
"loss_weights": loss_weights,
"loss_types": loss_types,
"input_dimention": input_dimention,
"normalization_mode": normalization_mode,
"mode": mode,
"batch_size": batch_size,
"gpuid": gpuid,
"gpu_limit": gpu_limit
}


In [20]:
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils
tf_model = keras.Sequential()

for layer in model.layers:
    tf_model.add(layer)
print(tf_model.summary())

ValueError: A merge layer should be called on a list of inputs.

In [8]:
print(args)

{'input_hdf5': None, 'input_testset': None, 'input_model': None, 'output_name': None, 'detection_threshold': 0.2, 'P_threshold': 0.1, 'S_threshold': 0.1, 'number_of_plots': 100, 'estimate_uncertainty': True, 'number_of_sampling': 5, 'loss_weights': [0.05, 0.4, 0.55], 'loss_types': ['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy'], 'input_dimention': (6000, 3), 'normalization_mode': 'std', 'mode': 'generator', 'batch_size': 500, 'gpuid': None, 'gpu_limit': None}


In [ ]:
# ================================================ #
#                  Pretrained-Model                #
# ================================================ #
#model = keras.models.load_model("pretrained/EqT_model.h5",custom_objects={'SeqSelfAttention':SeqSelfAttention})
args['input_model'] = "/Users/hao/opt/anaconda3/envs/eqtdev/lib/python3.7/site-packages/EQTransformer/pretrained/EqT_model.h5"

model = load_model(args['input_model'], custom_objects={'SeqSelfAttention': SeqSelfAttention,
                                                     'FeedForward': FeedForward,
                                                     'LayerNormalization': LayerNormalization,
                                                     'f1': f1
                                                     })



#frozon layers
for layer in model.layers[:-3]:
    layer.trainable = False
model.compile(loss = args['loss_types'],
              loss_weights =  args['loss_weights'],
              optimizer = Adam(lr = 0.001),
              metrics = [f1])
print(model.summary())
print('Loading is complete!', flush=True)  

In [ ]:
pbar_test = tqdm(total= int(np.ceil(len(test)/args['batch_size'])))
for _ in range(int(np.ceil(len(test) / args['batch_size']))):
    pbar_test.update()
    new_list = next(list_generator)

    if args['mode'].lower() == 'pre_load_generator':
        params_test = {'dim': args['input_dimention'][0],
                       'batch_size': len(new_list),
                       'n_channels': args['input_dimention'][-1],
                       'norm_mode': args['normalization_mode']}
        test_set={}
        fl = h5py.File(args['input_hdf5'], 'r')
        for ID in new_list:
            if ID.split('_')[-1] != 'NO':
                dataset = fl.get('data/'+str(ID))
            elif ID.split('_')[-1] == 'NO':
                dataset = fl.get('data/'+str(ID))
            test_set.update( {str(ID) : dataset})

        test_generator = PreLoadGeneratorTest(new_list, test_set, **params_test)

In [ ]:
pbar_test = tqdm(total= int(np.ceil(len(prediction_list)/args['batch_size'])), ncols=100, file=sys.stdout)        
for bn in range(int(np.ceil(len(prediction_list) / args['batch_size']))):  
    with nostdout():              
        pbar_test.update()

    new_list = next(list_generator)  
    prob_dic=_gen_predictor(new_list, args, model)

    pred_set={}
    for ID in new_list:
        dataset = fl.get('data/'+str(ID))
        pred_set.update( {str(ID) : dataset})  

    plt_n, detection_memory= _gen_writer(new_list, args, prob_dic, pred_set, HDF_PROB, predict_writer, save_figs, csvPr_gen, plt_n, detection_memory, keepPS, spLimit)   

In [27]:
print(model.layers[-1].name)
print(model.layers[-2].name)
print(model.layers[-3].name)
print(model.layers[-4].name)
print(model.layers[-5].name)
print(model.layers[-6].name)

picker_S
picker_P
detector
conv1d_45
conv1d_38
conv1d_31


In [26]:
# access inputs, outputs for model
# access input + output for layer
# access all layers
inputs = model.inputs
outputs = model.outputs
print(inputs)
print(outputs)

input0 = model.layers[0].input
output0 = model.layers[0].output
print(input0)
print(output0)


[<tf.Tensor 'input_4:0' shape=(None, 6000, 3) dtype=float32>]
[<tf.Tensor 'detector_4/Sigmoid:0' shape=(None, 6000, 1) dtype=float32>, <tf.Tensor 'picker_P_4/Sigmoid:0' shape=(None, None, 1) dtype=float32>, <tf.Tensor 'picker_S_4/Sigmoid:0' shape=(None, None, 1) dtype=float32>]
Tensor("input_4:0", shape=(None, 6000, 3), dtype=float32)
Tensor("input_4:0", shape=(None, 6000, 3), dtype=float32)


In [31]:
from keras.layers import add, Activation, LSTM, Conv1D
# Example: Transfer Learning:
base_model = model

print(base_model.summary())
decoder_D = base_model.layers[-6].output
decoder_P = base_model.layers[-5].output
decoder_S = base_model.layers[-4].output
x_S = base_model.layers[-4].output




Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 3000, 16)     1168        max_pooling1d_1[0][0]            
____________________________________________________________________________________________

In [32]:
d = Conv1D(1, 11, padding = 'same', activation='sigmoid', name='detector')(decoder_D)
P = Conv1D(1, 11, padding = 'same', activation='sigmoid', name='picker_P')(decoder_P)
S = Conv1D(1, 11, padding = 'same', activation='sigmoid', name='picker_S')(decoder_S)
Sg = Conv1D(1, 11, padding = 'same', activation='sigmoid', name='picker_Sg')(x_S)
new_outputs = [d, P, S, Sg]
new_model = keras.Model(inputs=base_model.inputs, outputs=new_outputs)
print(new_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 3000, 16)     1168        max_pooling1d_1[0][0]            
____________________________________________________________________________________________

In [35]:
for layer in new_model.layers[:-4]:
    layer.trainable = False
print(new_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 3000, 16)     1168        max_pooling1d_1[0][0]            
____________________________________________________________________________________________

In [52]:
args['padding'] = 'same'
args['activation'] = 'relu'
args['cnn_blocks'] = 5
args['lstm_blocks'] = 2
args['drop_rate'] =0.1
args['multi_gpu'] = False
args['number_of_gpus'] = 4

In [53]:
from keras.layers import Input
from EqT_utils import DataGenerator, _lr_schedule, cred2, PreLoadGenerator, data_reader
# ================================================ #
#                  Pretrained-Model                #
# ================================================ #   
inp = Input(shape=args['input_dimention'], name='input') 
model = cred2(nb_filters=[8, 16, 16, 32, 32, 64, 64],
          kernel_size=[11, 9, 7, 7, 5, 5, 3],
          padding=args['padding'],
          activationf =args['activation'],
          cnn_blocks=args['cnn_blocks'],
          BiLSTM_blocks=args['lstm_blocks'],
          drop_rate=args['drop_rate'], 
          loss_weights=args['loss_weights'],
          loss_types=args['loss_types'],
          kernel_regularizer=keras.regularizers.l2(1e-6),
          bias_regularizer=keras.regularizers.l1(1e-4),
          multi_gpu=args['multi_gpu'], 
          gpu_number=args['number_of_gpus'],  
           )(inp)  
print(model.summary())

Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 3000, 16)     1168        max_pooling1d_1[0][0]            
______________________________________________________________________

In [54]:
from keras.models import load_model
model.load_weights(args['input_model'])
print(model.summary())

ValueError: You are trying to load a weight file containing 80 layers into a model with 77 layers.

In [2]:
output_list = ['d','P','S']
if 'd' in output_list or 'D' in output_list:
    print('d')

d


In [3]:
for picker_name in output_list:
    if picker_name[0].upper() == 'P':
        print(picker_name)

P


In [19]:
# ================================================ #
#                  Transfer Learning               #
# ================================================ #
args['input_model'] = "/Users/hao/opt/anaconda3/envs/eqtdev/lib/python3.7/site-packages/EQTransformer/pretrained/EqT_model.h5"
new_model = load_model(args['input_model'], custom_objects={'SeqSelfAttention': SeqSelfAttention,
                                                     'FeedForward': FeedForward,
                                                     'LayerNormalization': LayerNormalization,
                                                     'f1': f1
                                                     })
#Hao     
#frozon layers except the output channels
for layer in new_model.layers[:-3]:
    layer.trainable = False

In [20]:
import keras
base_model = new_model    
new_outputs = []
decoder_D = base_model.layers[-6].output
decoder_P = base_model.layers[-5].output
decoder_S = base_model.layers[-4].output
if 'd' in output_list or 'D' in output_list:
    # add detector channel
    output_name = 'detector'
    d = Conv1D(1, 11, padding = 'same', activation='sigmoid', name=output_name)(decoder_D)
    new_outputs.append(d)
for picker_name in output_list:
    if picker_name[0].upper() == 'P':
        # add P-type output channel
        output_name = 'picker_' + picker_name
        P = Conv1D(1, 11, padding = 'same', activation='sigmoid', name=output_name)(decoder_P)
        new_outputs.append(P)
    if picker_name[0].upper() == 'S':
        # add S-type output channel
        output_name = 'picker_' + picker_name
        S = Conv1D(1, 11, padding = 'same', activation='sigmoid', name=output_name)(decoder_S)
        new_outputs.append(S)    
model = keras.Model(inputs=base_model.inputs, outputs=new_outputs)
print(model.summary())    

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 6000, 3)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 6000, 8)      272         input[0][0]                      
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 3000, 8)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 3000, 16)     1168        max_pooling1d_1[0][0]            
____________________________________________________________________________________________

In [18]:
output_list = ['Pn','Sg']